In [1]:
# === Cell 1: Setup - File Loading

%load_ext autoreload
%autoreload 2

import os
import time

print("Loading Sage/Python files...")

load('Packages/balancefunctions.sage') 

load('objects-invariants-properties/gt_precomputed_database.sage') 

load('objects-invariants-properties/gt.sage')

load('Packages/conjecturing.py')
load('Packages/graph_theory.py') 

try:
    from worker_funcs import _compute_invariant_value_worker, _compute_property_value_worker
    print("Worker functions for multiprocessing imported from worker_funcs.py.")
except ImportError:
    print("NOTE: worker_funcs.py not found or functions not imported.")
    print("      If gt_precomputed_database.sage uses multiprocessing, it might encounter issues.")
except Exception as e:
    print(f"Error importing from worker_funcs.py: {type(e).__name__}: {e}")


print("-" * 30)
print("Finished attempting to load all script files.")

Loading Sage/Python files...
Successfully imported worker functions into gt_precomputed_database.sage.
Database utility functions defined and cleaned up for Python 3.


WORKER (pid 44266): Loading /Users/liamdyer/Documents/Program Save Files/VSCode Save Files/Graphing Project/Packages/balancefunctions.sage...
WORKER (pid 44266): Loading /Users/liamdyer/Documents/Program Save Files/VSCode Save Files/Graphing Project/Packages/graph_theory.py...
WORKER (pid 44266): Finished loading dependency scripts.


loaded utilities
loaded invariants
loaded properties
loaded theorems
loaded graphs

Remember to load DIMACS and Sloane graphs if you want them
Worker functions for multiprocessing imported from worker_funcs.py.
------------------------------
Finished attempting to load all script files.


In [2]:
#Temp cell to test
print('update_invariant_database' in globals())
print('create_tables' in globals())

True
True


In [3]:
# Cell 2: Prepare Objects and Invariants

objects_to_study = []
n_val = 6
print(f"Generating graphs for n={n_val} using Sage's graphs({n_val})...")
try:
    objects_to_study.extend(list(graphs(n_val))) 
except NameError:
    print("ERROR: Sage's 'graphs()' generator not found.")
except Exception as e:
    print(f"An error occurred generating graphs: {e}")

if 'all_graphs' in globals() and isinstance(all_graphs, list) and all_graphs:
    num_graphs_from_gt = min(5, len(all_graphs)) 
    print(f"Adding first {num_graphs_from_gt} graphs from 'all_graphs' (loaded from gt.sage).")
    objects_to_study.extend(list(all_graphs[:num_graphs_from_gt])) 
else:
    print("NOTE: 'all_graphs' (from gt.sage) not found/empty. Using only graphs({n_val}).")

if objects_to_study:
    try:
        objects_to_study = sage.graphs.graph_list.unique_graphs(objects_to_study)
        print(f"Total unique graphs (objects) to study: {len(objects_to_study)}")
    except Exception as e_unique: # Catching general exception for unique_graphs
        print(f"Note: unique_graphs call failed or skipped ({e_unique}). Proceeding with potentially duplicate graphs.")
        print(f"Graphs to study: {len(objects_to_study)}")
else:
    print("WARNING: 'objects_to_study' is empty!")

prospective_invariants = []

# from balancefunctions.sage
my_primary_invariant_names = [
    'balance_number', 'balance_number_open',
    'graph_order', 'size', 'diameter', 
    'chromatic_number', 'is_connected', 
    'average_distance', 'girth', 'radius', 
    'wiener_index', 'szeged_index', 'average_degree'
]
print("\nLoading your primary invariants (from balancefunctions.sage):")
for name in my_primary_invariant_names:
    if name in globals():
        prospective_invariants.append(globals()[name])
        print(f"  Will use: '{name}'")
    else:
        print(f"  WARNING: Your primary function '{name}' not found in notebook's globals. Ensure definition and load.")

# Adds functions from Packages/graph_theory.py's 'invariants' list
if 'invariants' in globals() and isinstance(invariants, list):
    print(f"\nConsidering functions from graph_theory.py's 'invariants' list ({len(invariants)} items):")
    for func_from_gt in invariants:
        if callable(func_from_gt) and hasattr(func_from_gt, '__name__'):
            func_name = func_from_gt.__name__
            if func_name == 'lovasz_theta':
                print(f"  Explicitly skipping '{func_name}'.")
                continue
            # It will be added here; deduplication later will handle if your primary list already has it.
            prospective_invariants.append(func_from_gt)
            # print(f"  Temporarily adding '{func_name}' from graph_theory.py.") # Can be verbose
        elif callable(func_from_gt):
            prospective_invariants.append(func_from_gt) # Add unnamed callables
            # print(f"  Temporarily adding unnamed callable from graph_theory.py: {str(func_from_gt)[:60]}...")
else:
    print("\nWarning: Global 'invariants' list from graph_theory.py not found or not a list.")

# --- Deduplicate final_invariants_list, prioritizing those added first (i.e., your primary ones) ---
final_invariants_list = []
seen_names = set()
for func_obj in prospective_invariants:
    if callable(func_obj) and hasattr(func_obj, '__name__'):
        if func_obj.__name__ not in seen_names:
            final_invariants_list.append(func_obj)
            seen_names.add(func_obj.__name__)
    elif callable(func_obj): # For unnamed callables, add if not identical object already present
        is_present = False
        for existing_func in final_invariants_list:
            if func_obj is existing_func: # Check object identity
                is_present = True
                break
        if not is_present:
            final_invariants_list.append(func_obj)


# --- Print the final list ---
print(f"\nTotal number of unique invariants to compute: {len(final_invariants_list)}")
print("Final Invariants whose names will be passed to worker processes:")
for inv_func in final_invariants_list:
    func_name = inv_func.__name__ if hasattr(inv_func, '__name__') else f"Unnamed callable: {str(inv_func)[:60]}..."
    print(f"  - {func_name}")

print("-" * 30)
print("Cell 2 (Prepare Objects and Invariants) finished.")

Generating graphs for n=6 using Sage's graphs(6)...
Adding first 5 graphs from 'all_graphs' (loaded from gt.sage).
Note: unique_graphs call failed or skipped (module 'sage.graphs.graph_list' has no attribute 'unique_graphs'). Proceeding with potentially duplicate graphs.
Graphs to study: 161

Loading your primary invariants (from balancefunctions.sage):
  Will use: 'balance_number'
  Will use: 'balance_number_open'
  Will use: 'graph_order'
  Will use: 'size'
  Will use: 'diameter'
  Will use: 'chromatic_number'
  Will use: 'is_connected'
  Will use: 'average_distance'
  Will use: 'girth'
  Will use: 'radius'
  Will use: 'wiener_index'
  Will use: 'szeged_index'
  Will use: 'average_degree'

Considering functions from graph_theory.py's 'invariants' list (18 items):
  Explicitly skipping 'lovasz_theta'.

Total number of unique invariants to compute: 22
Final Invariants whose names will be passed to worker processes:
  - balance_number
  - balance_number_open
  - graph_order
  - size
  -

In [ ]:
# Cell 2.5: Pre-compute Invariants w/ Progress Updates

# This cell assumes that 'objects_to_study' and 'final_invariants_list' 
# were successfully defined and populated in the preceding cell (Cell 2).

# It also assumes that 'create_tables' and 'update_invariant_database' 
# were successfully loaded from your database utility script (e.g., gt_precomputed_database.sage) in Cell 1.

INVARIANTS_DB_FILENAME = "invariants.db" 
CONJECTURES_OUT_FILENAME = "conjectures_output.txt"

# --- Check if necessary components are ready ---
proceed_with_precomputation = True
if 'update_invariant_database' not in globals() or 'create_tables' not in globals():
    print("WARNING: 'update_invariant_database' or 'create_tables' function not found.")
    print("         These are expected from your database utility script (e.g., gt_precomputed_database.sage).")
    print("         Progress updates for invariant calculation will NOT be shown separately by this cell.")
    print("         The 'conjecture' function in the next cell will compute invariants internally.")
    proceed_with_precomputation = False

if 'objects_to_study' not in globals() or not objects_to_study:
    print("WARNING: 'objects_to_study' list is empty or not defined. Cannot pre-compute.")
    proceed_with_precomputation = False
    
if 'final_invariants_list' not in globals() or not final_invariants_list:
    print("WARNING: 'final_invariants_list' is empty or not defined. Cannot pre-compute.")
    proceed_with_precomputation = False

# --- Start Pre-computation ---
if proceed_with_precomputation:
    print(f"\nCreating tables in '{INVARIANTS_DB_FILENAME}' (if they don't exist)...")
    try:
        create_tables(database_file=INVARIANTS_DB_FILENAME) 
        print(f"Successfully ensured tables exist in '{INVARIANTS_DB_FILENAME}'.")
        
        print(f"\nPre-computing and storing {len(final_invariants_list)} invariants for {len(objects_to_study)} graphs in '{INVARIANTS_DB_FILENAME}'...")
        print("This may take a significant amount of time. Progress updates will follow.")
        
        # Ensure worker_funcs.py was imported in Cell 1 if your update_invariant_database uses them
        # for multiprocessing without the AttributeError for _compute_..._worker.
        if ('_compute_invariant_value_worker' not in globals() and 
            '_compute_property_value_worker' not in globals() and 
            WORKER_FUNCS_DEFINED is False): # WORKER_FUNCS_DEFINED was from my example Cell 1
             print("Warning: Multiprocessing worker functions may not be correctly set up if imported from worker_funcs.py and that import failed.")

        update_invariant_database(
            invariants_list=final_invariants_list, 
            graphs_list=objects_to_study, 
            timeout=600,  # Timeout per individual invariant/graph computation in seconds (e.g., 10 minutes)
                          # Adjust as needed for your balance_number complexity and n_val
            database_file=INVARIANTS_DB_FILENAME, # Ensure it writes to the DB conjecturing.py will use
            verbose=False  # progress prints
        )
        print(f"\nFinished pre-computing and storing invariants into '{INVARIANTS_DB_FILENAME}'.")
        print("You can now proceed to Cell 3 to run the conjecturing engine using these precomputed values.")
        
    except NameError as ne:
        print(f"\nA NameError occurred during pre-computation setup: {ne}. ")
        print("This usually means 'update_invariant_database' or 'create_tables' was not successfully loaded in Cell 1.")
    except Exception as e:
        print(f"\nAn unexpected error occurred during the pre-computation of invariants: {type(e).__name__}: {e}")
else:
    print("\nSkipping invariant pre-computation due to missing functions or empty object/invariant lists.")

print("-" * 30)
print("Cell 2.5 (Pre-compute Invariants) finished.")


Creating tables in 'invariants.db' (if they don't exist)...
Successfully ensured tables exist in 'invariants.db'.

Pre-computing and storing 22 invariants for 161 graphs in 'invariants.db'...
This may take a significant amount of time. Progress updates will follow.


DEBUG: Loaded 4 entries from skip list: skip_timeouts_log.txt


  Dispatching computation: balance_number for graph E??? (E???)...


WORKER (pid 44480): Loading /Users/liamdyer/Documents/Program Save Files/VSCode Save Files/Graphing Project/Packages/balancefunctions.sage...


Stored balance_number for E???: 6.0
  Dispatching computation: balance_number for graph E??G (E??G)...


...

Stored dominationNumber for E@^o: 2.0
  Dispatching computation: dominationNumber for graph E@\w (E@\w)...


WORKER (pid 51666): Loading /Users/liamdyer/Documents/Program Save Files/VSCode Save Files/Graphing Project/Packages/balancefunctions.sage...
WORKER (pid 51666): Loading /Users/liamdyer/Documents/Program Save Files/VSCode Save Files/Graphing Project/Packages/graph_theory.py...
WORKER (pid 51666): Finished loading dependency scripts.


Stored dominationNumber for E@\w: 2.0
  Dispatching computation: dominationNumber for graph E@]w (E@]w)...


WORKER (pid 51679): Loading /Users/liamdyer/Documents/Program Save Files/VSCode Save Files/Graphing Project/Packages/balancefunctions.sage...
WORKER (pid 51679): Loading /Users/liamdyer/Documents/Program Save Files/VSCode Save Files/Graphing Project/Packages/graph_theory.py...
WORKER (pid 51679): Finished loading dependency scripts.


Stored dominationNumber for E@]w: 2.0
  Dispatching computation: dominationNumber for graph E@^w (E@^w)...


In [ ]:
# Cell to inspect invariants.db using your existing functions

# Ensure os module is available (from sage.all or explicit import)
import os 

# INVARIANTS_DB_FILENAME should be defined from your Cell 2.5 (e.g., "invariants.db")
# Make sure 'invariants_as_dict' function is available (loaded from gt_precomputed_database.sage in Cell 1)

if 'invariants_as_dict' in globals() and \
   'INVARIANTS_DB_FILENAME' in globals() and \
   os.path.exists(INVARIANTS_DB_FILENAME):

    print(f"Attempting to load data from '{INVARIANTS_DB_FILENAME}' using invariants_as_dict()...")
    all_invariant_data = invariants_as_dict(database_file=INVARIANTS_DB_FILENAME)

    if all_invariant_data:
        print(f"Successfully loaded data for {len(all_invariant_data)} graphs from '{INVARIANTS_DB_FILENAME}'.")

        # Print data for the first graph (or a few graphs) as a sample
        graphs_to_sample = list(all_invariant_data.keys())[:min(3, len(all_invariant_data))] # Sample up to 3 graphs

        for graph_key in graphs_to_sample:
            print(f"\nSample data for graph (g6 string): '{graph_key}':")
            if isinstance(all_invariant_data[graph_key], dict):
                for inv_name, value in all_invariant_data[graph_key].items():
                    # Check for error strings that your worker might have stored
                    if isinstance(value, str) and value.lower().startswith("error:"):
                        print(f"  {inv_name}: {value} (This indicates an error during computation)")
                    else:
                        print(f"  {inv_name}: {value} (Type: {type(value)})")
            else:
                print(f"  Unexpected data format for graph {graph_key}: {all_invariant_data[graph_key]}")

        if len(all_invariant_data) > len(graphs_to_sample):
            print(f"\n... and data for {len(all_invariant_data) - len(graphs_to_sample)} more graphs.")

        # You can add more code here to inspect specific graphs or invariants
        # For example, to see all data for a specific graph's g6 string:
        # specific_g6 = "Dqg" # graph6 for C5, replace with a g6 from your DB
        # if specific_g6 in all_invariant_data:
        #    print(f"\nData for graph '{specific_g6}': {all_invariant_data[specific_g6]}")

    else:
        print(f"'{INVARIANTS_DB_FILENAME}' seems to be empty or no data was loaded by invariants_as_dict.")
        print("This might happen if Cell 2.5 (pre-computation) didn't store any successful results.")
else:
    if 'invariants_as_dict' not in globals():
        print(f"ERROR: 'invariants_as_dict' function not found. Ensure 'gt_precomputed_database.sage' is loaded.")
    if 'INVARIANTS_DB_FILENAME' not in globals():
        print(f"ERROR: 'INVARIANTS_DB_FILENAME' variable not defined. Ensure Cell 2.5 was run or define it.")
    elif not os.path.exists(INVARIANTS_DB_FILENAME):
        print(f"ERROR: Database file '{INVARIANTS_DB_FILENAME}' does not exist. Ensure Cell 2.5 ran and created it.")

In [ ]:
# Quick check cell
if 'invariants_as_dict' in globals() and os.path.exists(INVARIANTS_DB_FILENAME):
    db_content = invariants_as_dict(INVARIANTS_DB_FILENAME)
    if db_content:
        print(f"'{INVARIANTS_DB_FILENAME}' contains {len(db_content)} graphs with precomputed invariants.")
        # print(list(db_content.items())[:1]) # Print first item as sample
    else:
        print(f"'{INVARIANTS_DB_FILENAME}' is empty or could not be read properly by invariants_as_dict.")
else:
    print(f"'{INVARIANTS_DB_FILENAME}' not found or invariants_as_dict not loaded.")

In [ ]:
# === Cell 3: Run Conjecturing (Corrected) ===

# IMPORTANT: 
# - The 'expressions' C program must be in the same directory as this notebook.
# - If Cell 2.5 ran successfully, conjecturing.py should use the values from invariants.db.

if not objects_to_study:
    print("No objects to study, skipping conjecture generation.")
elif not final_invariants_list:
    print("No invariants defined, skipping conjecture generation.")
elif 'conjecture' not in globals():
    print("ERROR: The 'conjecture' function (from conjecturing.py) is not defined. Check Cell 1 loads.")
else:
    # --- CHOOSE YOUR MAIN INVARIANT ---
    # This is the invariant you want to find conjectures FOR (e.g., bounds on balance_number).
    # It must be one of the functions in your final_invariants_list.
    # Let's try to find 'balance_number' in your list.
    
    main_invariant_function_to_bound = None
    main_invariant_name_to_bound = "balance_number" # <<<< CHANGE THIS if you want to bound a different invariant

    main_invariant_idx = -1
    for idx, func in enumerate(final_invariants_list):
        if hasattr(func, '__name__') and func.__name__ == main_invariant_name_to_bound:
            main_invariant_function_to_bound = func
            main_invariant_idx = idx
            break
            
    if main_invariant_function_to_bound is None:
        print(f"ERROR: Could not find the main invariant '{main_invariant_name_to_bound}' in your final_invariants_list.")
        print(f"Available invariant names: {[f.__name__ for f in final_invariants_list if hasattr(f, '__name__')]}")
        print("Skipping conjecture generation.")
    else:
        print(f"Chosen main invariant for conjecturing: '{main_invariant_function_to_bound.__name__}' (at index {main_invariant_idx} in your list).")

        # --- Prepare precomputed data (if Cell 2.5 ran and created invariants.db) ---
        use_precomputed_flag = False
        precomputed_data_tuple_for_conjecture = None
        if 'precomputed_invariants_for_conjecture' in globals() and \
           'INVARIANTS_DB_FILENAME' in globals() and \
           os.path.exists(INVARIANTS_DB_FILENAME):
            try:
                precomputed_data_tuple_for_conjecture = precomputed_invariants_for_conjecture(database_file=INVARIANTS_DB_FILENAME)
                use_precomputed_flag = True
                print(f"Will attempt to use precomputed invariants from '{INVARIANTS_DB_FILENAME}'.")
            except Exception as e_prec:
                print(f"Could not prepare precomputed data for conjecture function: {e_prec}")
                print("Conjecture function will compute invariants on the fly (this will be slow).")
        else:
            db_file_to_check = INVARIANTS_DB_FILENAME if 'INVARIANTS_DB_FILENAME' in globals() else "invariants.db (default name)"
            print(f"Precomputation data function ('precomputed_invariants_for_conjecture') not found, "
                  f"or database ('{db_file_to_check}') missing.")
            print("Conjecture function will compute invariants on the fly (this will be slow).")

        # --- Set parameters for calling the 'expressions' C program ---
        expressions_time_limit = 30  # Seconds per main invariant. Increase if needed for deeper search.
        debug_expressions = False    # Set to True for C program's stderr (fix prints in conjecturing.py first)
        verbose_expressions = False  # Set to True (with debug=True) for C program's verbose stdout (fix prints)

        # --- Strategy 1: Find Lower Bounds (main_invariant >= Expression) ---
        print(f"\nStarting conjecture generation for LOWER BOUNDS on '{main_invariant_function_to_bound.__name__}'...")
        print(f"(Looking for: {main_invariant_function_to_bound.__name__}(G) >= SomeFormula)")
        print("This part involves the 'expressions' C program and may take some time...")
        try:
            returned_conjectures_lower = conjecture(
                objects_to_study, 
                final_invariants_list, 
                main_invariant_idx,               # The index of the invariant we are bounding
                upperBound=False,                 # False for lower bounds (main_inv >= X)
                precomputed=precomputed_data_tuple_for_conjecture if use_precomputed_flag else None,
                expressions_timeout=expressions_time_limit,
                debug=debug_expressions, 
                verbose=verbose_expressions
            ) 
            
            print(f"\nReturn value from conjecture(lower bounds): {returned_conjectures_lower}")
            if returned_conjectures_lower and isinstance(returned_conjectures_lower, list) and returned_conjectures_lower:
                print(f"Conjectures found (Lower Bounds for {main_invariant_function_to_bound.__name__}):")
                with open(CONJECTURES_OUT_FILENAME, "a") as f_out: # Open file in append mode
                    f_out.write(f"\n# Lower Bounds for {main_invariant_function_to_bound.__name__} on {time.strftime('%Y-%m-%d %H:%M:%S')}\n")
                    for conj_obj in returned_conjectures_lower:
                        conj_str = str(conj_obj)
                        print(f"  {conj_str}")
                        f_out.write(f"  {conj_str}\n")
                print(f"Also saved to ", CONJECTURES_OUT_FILENAME)
            else:
                print("No lower bound conjectures were returned directly as a non-empty list.")
                with open(CONJECTURES_OUT_FILENAME, "a") as f_out: # Open file in append mode
                    f_out.write(f"\n# No Lower Bounds found for {main_invariant_function_to_bound.__name__} on {time.strftime('%Y-%m-%d %H:%M:%S')}\n")
                    
        except Exception as e:
            print(f"An ERROR occurred during lower bound conjecture generation: {e}")

        print("-" * 40)

        # --- Strategy 2: Find Upper Bounds (main_invariant <= Expression) ---
        print(f"\nStarting conjecture generation for UPPER BOUNDS on '{main_invariant_function_to_bound.__name__}'...")
        print(f"(Looking for: {main_invariant_function_to_bound.__name__}(G) <= SomeFormula)")
        print("This part involves the 'expressions' C program and may take some time...")
        try:
            returned_conjectures_upper = conjecture(
                objects_to_study, 
                final_invariants_list, 
                main_invariant_idx,               # The index of the invariant we are bounding
                upperBound=True,                  # True for upper bounds (main_inv <= X)
                precomputed=precomputed_data_tuple_for_conjecture if use_precomputed_flag else None,
                expressions_timeout=expressions_time_limit,
                debug=debug_expressions,
                verbose=verbose_expressions
            )
            
            print(f"\nReturn value from conjecture(upper bounds): {returned_conjectures_upper}")
            if returned_conjectures_upper and isinstance(returned_conjectures_upper, list) and returned_conjectures_upper:
                print(f"Conjectures found (Upper Bounds for {main_invariant_function_to_bound.__name__}):")
                with open(CONJECTURES_OUT_FILENAME, "a") as f_out: # Open file in append mode
                    f_out.write(f"\n# Upper Bounds for {main_invariant_function_to_bound.__name__} on {time.strftime('%Y-%m-%d %H:%M:%S')}\n")
                    for conj_obj in returned_conjectures_upper:
                        conj_str = str(conj_obj)
                        print(f"  {conj_str}")
                        f_out.write(f"  {conj_str}\n")
                print(f"Also saved to", CONJECTURES_OUT_FILENAME)
            else:
                print("No upper bound conjectures were returned directly as a non-empty list.")
                with open(CONJECTURES_OUT_FILENAME, "a") as f_out: # Open file in append mode
                    f_out.write(f"\n# No Upper Bounds found for {main_invariant_function_to_bound.__name__} on {time.strftime('%Y-%m-%d %H:%M:%S')}\n")
                
        except Exception as e:
            print(f"An ERROR occurred during upper bound conjecture generation: {e}")

        print("-" * 40)
        print("Conjecture generation attempts finished.")